In [1]:
from pathlib import Path
import numpy as np
from collections import Counter
import json
root = Path("data/extracted")
num = 90
t_len = 5000
label_set = set()
ds_list = ["gpts"]
for ds in ds_list:
    path = root / (ds+".npz")
    data = np.load(path, allow_pickle=True)
    c = Counter(data["labels"])
    enough_labels = {k for k,v in c.items() if v >= num}
    label_set = label_set & enough_labels if label_set else enough_labels

In [32]:
len(label_set)

100

In [28]:
label2idx = {k:i for i,k in enumerate(label_set)}
json.dump(label2idx, open("data/wflibdata/gpts.json","w"))
for ds in ds_list:
    path = root / (ds+".npz")
    data = np.load(path, allow_pickle=True)
    traces,labels = data["traces"],data["labels"]
    final_indices = []
    for label in label_set:
        indices = np.where(labels==label)[0]
        np.random.shuffle(indices)
        final_indices.extend(indices[:100])
    ds_size = len(final_indices)
    X = np.zeros((ds_size,t_len))
    y = np.zeros(ds_size)
    for i, idx in enumerate(final_indices):
        t = traces[idx]
        trace = (t[:,0]-t[0,0]+1)*t[:,1]
        X[i] = trace[:t_len]if len(trace) >= t_len else np.pad(trace,(0,t_len-len(trace)))
        y[i] = label2idx[labels[idx]]
    np.savez(Path("data/wflibdata")/(ds+"1.npz"),X=X,y=y)

In [20]:
label2idx = {k:i for i,k in enumerate(label_set)}
json.dump(label2idx, open("data/wflibdata/gpts_full_info.json","w"))
for ds in ds_list:
    path = root / (ds+".npz")
    data = np.load(path, allow_pickle=True)
    traces,labels = data["traces"],data["labels"]
    final_indices = []
    for label in label_set:
        indices = np.where(labels==label)[0]
        np.random.shuffle(indices)
        final_indices.extend(indices[:100])
    ds_size = len(final_indices)
    X_ts = np.zeros((ds_size,t_len),dtype=np.float32)
    X_dir = np.zeros((ds_size,t_len),dtype=np.int32)
    X_size = np.zeros((ds_size,t_len),dtype=np.int32)
    y = np.zeros(ds_size)
    for i, idx in enumerate(final_indices):
        t = traces[idx]
        trace_ts = t[:,0]-t[0,0]
        X_ts[i] = trace_ts[:t_len]if len(trace_ts) >= t_len else np.pad(trace_ts,(0,t_len-len(trace_ts)))
        trace_dir = t[:,1]
        X_dir[i] = trace_dir[:t_len]if len(trace_dir) >= t_len else np.pad(trace_dir,(0,t_len-len(trace_dir)))
        trace_size = t[:,2]
        X_size[i] = trace_size[:t_len] if len(trace_size) >= t_len else np.pad(trace_size,(0,t_len-len(trace_dir)))
        y[i] = label2idx[labels[idx]]
    np.savez(Path("data/wflibdata")/(ds+"_full_info.npz"),ts=X_ts,dir=X_dir,size=X_size,y=y)
    np.savez(Path("data/wflibdata")/(ds+"_combined.npz"),X=np.stack([X_ts,X_dir,X_size],axis=-1),y=y)

In [50]:
data=np.load(Path("data/wflibdata")/(ds+"_full_info.npz"))

In [22]:
np.stack([X_ts,X_dir,X_size],axis=-1)[0]

array([[ 0.00000000e+00,  1.00000000e+00,  1.73900000e+03],
       [ 1.73282623e-03, -1.00000000e+00,  5.75900000e+03],
       [ 9.23929214e-02,  1.00000000e+00,  1.78500000e+03],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])